## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,73.51,84,100,9.93,light rain
1,1,Hermanus,ZA,-34.4187,19.2345,52.81,80,19,7.72,light rain
2,2,Thompson,CA,55.7435,-97.8558,19.56,45,40,23.02,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,44.71,70,75,3.44,broken clouds
4,4,Hobart,AU,-42.8794,147.3294,54.27,90,100,8.05,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,73.51,84,100,9.93,light rain
5,5,Tual,ID,-5.6667,132.7500,83.89,66,82,4.54,broken clouds
6,6,Karratha,AU,-20.7377,116.8463,79.07,58,1,7.18,light rain
8,8,Jalu,LY,29.0331,21.5482,84.33,13,30,8.16,scattered clouds
10,10,Kapaa,US,22.0752,-159.3190,78.78,84,100,18.41,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
674,674,Tessalit,ML,20.1986,1.0114,85.66,7,0,10.18,clear sky
676,676,Ler,SS,8.3018,30.1418,83.80,58,100,9.62,overcast clouds
677,677,Port Blair,IN,11.6667,92.7500,80.64,83,20,2.30,mist
678,678,Poya,NC,-21.3500,165.1500,70.72,88,100,9.66,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.51,light rain,-23.1203,-134.9692,
5,Tual,ID,83.89,broken clouds,-5.6667,132.7500,
6,Karratha,AU,79.07,light rain,-20.7377,116.8463,
8,Jalu,LY,84.33,scattered clouds,29.0331,21.5482,
10,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,
12,Tomatlan,MX,89.11,clear sky,19.9333,-105.2500,
13,Tanete,ID,74.50,broken clouds,-3.9443,119.7848,
14,Wanning,CN,78.51,broken clouds,18.8003,110.3967,
15,Jamestown,US,82.53,clear sky,42.0970,-79.2353,
16,Salalah,OM,81.81,scattered clouds,17.0151,54.0924,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

hotel_list = []

# 6b. Iterate through the hotel DataFrame.

for i,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
       
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

     # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotels["results"][0]["name"]
    
        hotel_list.append(hotel_name)
        
    except:
        hotel_list.append(np.NaN)
        pass
    
hotel_df["Hotel Name"] = hotel_list  

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name<dd>{City}</dd></dt>
<dt>Country Code<dd>{Country}</dd></dt>
<dt>Weather Description<dd>{Current Description}</dd></dt>
<dt>Maximum Temperature<dd>{Max Temp}</dd></dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))